In [1]:
!pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=da554b12050ed079f6332391eb2daf4109412f7421bb0e2b4a3b9318fdc172c6
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf

In [ ]:
#Initializing Spark
conf = SparkConf().setAppName("RDD_practice").setMaster("local[*]") 
sc = SparkContext(conf=conf)

In [7]:
print(sc)

<SparkContext master=local[*] appName=testApp1>


In [8]:
#Lets see how many cores are available
sc.defaultParallelism

2

In [9]:
#Generate random data
import random
randomlist =  random.sample(range(0, 40),10)
print(randomlist)

[13, 8, 26, 35, 25, 34, 11, 21, 1, 12]


In [11]:
#Create RDD
#I want to split my data into 4 parts. We are going to use "collect" because we have small data.
rdd1 = sc.parallelize(randomlist, 4)
rdd1.collect()


[13, 8, 26, 35, 25, 34, 11, 21, 1, 12]

In [16]:
#Data distributions in partitions
rdd1.getNumPartitions()
#lets print all partitions
print(rdd1.glom().collect())
#lets print first 2 partitions
print(rdd1.glom().take(2))
#lets print last partition
print(rdd1.glom().collect()[3])

[[13, 8], [26, 35], [25, 34], [11, 21, 1, 12]]
[[13, 8], [26, 35]]
[11, 21, 1, 12]


In [17]:
#count
print(rdd1.count())

10


In [19]:
#first
print(rdd1.first())

13


In [20]:
#top
print(rdd1.top(2))

[35, 34]


In [22]:
#distinct
print(rdd1.distinct().collect())

[8, 12, 13, 25, 21, 1, 26, 34, 35, 11]


In [23]:
#map. map transformation returns a new rdd
rdd_map = rdd1.map(lambda item: (item + 3) * 3 )
print(rdd_map.collect())

[48, 33, 87, 114, 84, 111, 42, 72, 12, 45]


In [26]:
#filter. filter returns a new dataset
rdd_filter = rdd1.filter(lambda x: x%3 == 0)
print(rdd_filter.collect())

[21, 12]


In [28]:
#flatMap. flatmap is similar to map, but each item is can be mapped to 0 or more output items. It collects in to single list.
rdd_flatmap = rdd1.flatMap(lambda x: [ x+2, x+5 ])
print(rdd_flatmap.collect())
#lets use reduce action. 
rdd_flatmap.reduce(lambda x, y : x+y)


[15, 18, 10, 13, 28, 31, 37, 40, 27, 30, 36, 39, 13, 16, 23, 26, 3, 6, 14, 17]


442

In [32]:
#Descriptive statistics
print([rdd1.max(),rdd1.min(),rdd1.mean(),round(rdd1.stdev(), 2 ),rdd1.sum()])

[35, 1, 18.6, 10.78, 186]


In [35]:
#mapPartitions. We will have a specific calculation for each partitions.
def myfunc(partition):
  sum = 0
  for item in partition:
    sum = sum + item
  yield sum
#We can not use return here, because it returns only one result. Here we want one result per partition.
print(rdd1.mapPartitions(myfunc).collect())


[21, 61, 59, 45]
